# SimplifyUR: Unsupervised Lexical Text Simplification for Urdu

Let's start by loading in the packages.

In [1]:
from __future__ import division
import gensim
from gensim.models import Word2Vec
import codecs
from pickle import load
import collections
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
import unicodedata
import re
from collections import Counter
import sys
import copy

Execute the main functions that take the complex sentence as input and output its simplified form.

In [2]:
def handle_grammatical_case(backBackBack, backBack, back, word, forw, forwForw, forwForwForw, caseMarkers, lm):
    
    forwardWithProb = {}
    backwardWithProb = {}
    middleWithProb = {}
    if (backBack, back, word) in lm.keys():
        if (word, forw, forwForw) in lm.keys():
            if (back, word, forw) in lm.keys():
                return [back, forw]
            else:
                if forw in caseMarkers:
                    for fc in caseMarkers:
                        if (word, fc, forwForw) in lm.keys():
                            forwardWithProb[lm[(word, fc, forwForw)]] = \
                                (word, fc, forwForw)
                    sortForwProb = sorted(forwardWithProb.keys(),
                            reverse=True)
                    if forwardWithProb:
                        for f in sortForwProb:
                            if (word, forwardWithProb[f][1], forwForw) \
                                in lm.keys():
                                if (back, word, forwardWithProb[f][1]) \
                                    in lm.keys():

                                    return [back, forwardWithProb[f][1]]
                else:
                    return False
        else:
            if forw in caseMarkers:
                for fc in caseMarkers:
                    if (word, fc, forwForw) in lm.keys():
                        forwardWithProb[lm[(word, fc, forwForw)]] = \
                            (word, fc, forwForw)
                sortForwProb = sorted(forwardWithProb.keys(),
                        reverse=True)
                if forwardWithProb:
                    for f in sortForwProb:
                        if (word, forwardWithProb[f][1], forwForw) \
                            in lm.keys():
                            if (back, word, forwardWithProb[f][1]) \
                                in lm.keys():
                                return [back, forwardWithProb[f][1]]
            else:
                return False
        if forw in caseMarkers and (word, forwForw, forwForwForw) \
            in lm.keys():
            return [back, '']
        else:
            return False

    if back in caseMarkers:
        for c in caseMarkers:
            if (backBack, c, word) in lm.keys():
                backwardWithProb[lm[(backBack, c, word)]] = (backBack,
                        c, word)
        sortBackProb = sorted(backwardWithProb.keys(), reverse=True)
        if backwardWithProb:
            for b in sortBackProb:
                if (word, forw, forwForw) in lm.keys():
                    if (backwardWithProb[b][1], word, forw) \
                        in lm.keys():
                        return [backwardWithProb[b][1], forw]
                else:
                    if forw in caseMarkers:
                        for fc in caseMarkers:
                            if (word, fc, forwForw) in lm.keys():
                                forwardWithProb[lm[(word, fc,
                                        forwForw)]] = (word, fc,
                                        forwForw)
                        sortForwProb = sorted(forwardWithProb.keys(),
                                reverse=True)
                        if forwardWithProb:
                            for f in sortForwProb:
                                if (word, forwardWithProb[f][1],
                                    forwForw) in lm.keys():
                                    if (backwardWithProb[b][1], word,
        forwardWithProb[f][1]) in lm.keys():
                                        return [backwardWithProb[b][1],
        forwardWithProb[f][1]]
                if forw in caseMarkers and (word, forwForw,
                        forwForwForw) in lm.keys():
                    if (backwardWithProb[b][1], word, forwForw) \
                        in lm.keys():
                        return [backwardWithProb[b][1], '']
        if back in caseMarkers:
            if (backBackBack, backBack, word) in lm.keys():
                if (word, forw, forwForw) in lm.keys():
                    if (backBack, word, forw) in lm.keys():
                        return ['', forw]
                    else:
                        if forw in caseMarkers:
                            for fc in caseMarkers:
                                if (word, fc, forwForw) in lm.keys():
                                    forwardWithProb[lm[(word, fc,
        forwForw)]] = (word, fc, forwForw)
                            sortForwProb = \
                                sorted(forwardWithProb.keys(),
                                    reverse=True)
                            if forwardWithProb:
                                for f in sortForwProb:
                                    if (word, forwardWithProb[f][1],
        forwForw) in lm.keys():
                                        if (backBack, word,
        forwardWithProb[f][1]) in lm.keys():
                                            return ['',
        forwardWithProb[f][1]]
                else:
                    if forw in caseMarkers:
                        for fc in caseMarkers:
                            if (word, fc, forwForw) in lm.keys():
                                forwardWithProb[lm[(word, fc,
                                        forwForw)]] = (word, fc,
                                        forwForw)
                        sortForwProb = sorted(forwardWithProb.keys(),
                                reverse=True)
                        if forwardWithProb:
                            for f in sortForwProb:
                                if (word, forwardWithProb[f][1],
                                    forwForw) in lm.keys():
                                    if (backBack, word,
        forwardWithProb[f][1]) in lm.keys():
                                        return ['',
        forwardWithProb[f][1]]
                if forw in caseMarkers and (word, forwForw,
                        forwForwForw) in lm.keys():
                    return ['', '']
                else:
                    return False
    return False

In [3]:
def simplify(content, synModel, tagger, lm, sortedMaxToMinWords, dontRemove):

    final = []
    orig = []


    caseMarkers=["تک","تلک","پر","سی","سا","کےلۓ","کے لۓ","کے لیے",
                   "کےلیے","کیلیے","پاس","پار","ساتھ","سے","کے","کی","کو","کا","میں","نے","کیلئے"]

    temp = ""
    for sent in content:
        sent = unicodedata.normalize('NFC',sent)
        orig = sent.split(' ')
        li = sent.split(' ')
        liTag = tagger.tag(li)

        i = -1
        indexes = []
        for currentWord in liTag:
            i += 1
            if (currentWord[0] not in dontRemove) and (currentWord[1] == 'NN' or currentWord[1] == 'ADJ' or currentWord[1] == "VB" or currentWord[1] == "ADV" or currentWord[1] == "Q"):
                if currentWord[0] in synModel.wv.vocab:
                    n = synModel.wv.most_similar(positive=(currentWord[0]), topn = 10)#10)
                    index = []
                    for oneSyn in n:
                        if oneSyn[0] in sortedMaxToMinWords:
                            if oneSyn[0] not in currentWord[0] and currentWord[0] not in oneSyn[0] and currentWord[0][0:len(currentWord[0])-1] not in oneSyn[0] and oneSyn[0][0:len(oneSyn[0])-1] not in currentWord[0] and currentWord[0][1:len(currentWord[0])] not in oneSyn[0] and oneSyn[0][1:len(oneSyn[0])] not in currentWord[0]:
                                index.append(sortedMaxToMinWords.index(oneSyn[0]))
                    index = sorted(index)
                    replaceIndex = li.index(currentWord[0])

                    for ind in index:
                        backBackBack = ""
                        backBack = ""
                        back = ""
                        forw = ""
                        forwForw = ""
                        forwForwForw = ""

                        if i-3 in range(len(orig)):
                            backBackBack = orig[i-3]
                        else:
                            backBackBack = "<s>"
                        if i-2 in range(len(orig)):
                            backBack = orig[i-2]
                        else:
                            backBack = "<s>"
                        if i-1 in range(len(orig)):
                            back = orig[i-1]
                        else:
                            back = "<s>"
                        if i+3 in range(len(orig)):
                            forwForwForw = orig[i+3]
                        else:
                            forwForwForw = "</s>"
                        if i+2 in range(len(orig)):
                            forwForw = orig[i+2]
                        else:
                            forwForw = "</s>"
                        if i+1 in range(len(orig)):
                            forw = orig[i+1]
                        else:
                            forw = "</s>"
                        case = handle_grammatical_case(backBackBack,backBack,back,sortedMaxToMinWords[ind],forw,forwForw,forwForwForw,caseMarkers,lm)
                        if case != False:
                            orig[i] = sortedMaxToMinWords[ind]
                            if i-1 in range(len(orig)):
                                orig[i-1] = case[0]
                            if i+1 in range(len(orig)):
                                orig[i+1] = case[1]
                            break

        temp = str.join(' ',orig)
        final.append(re.sub(r"\s{2,}", " ", temp))
    return final

## Import necessary models and data files

Word2Vec Model: A CBOW model trained on different Urdu corpora (over 240 million words) with a window size of 5 words to obtain 300 dimensional vector representations of Urdu words. 

In [4]:
synModel  = Word2Vec.load("Models/extended.model")

Parts of Speech (PoS) Tagger: Trained on an Urdu corpus of 95.4 million tokens with 41 tags.

In [5]:
file = open('Models/trigramPOS.pkl', 'rb')
tagger = load(file)
file.close()

Language Model: A trigram language model trained on 120.9 million tokens.

In [6]:
file = open('Models/trigramsLM.pkl', 'rb')
lm = load(file)
file.close()

Common_Words.txt: A list of 370 frequently occuring Urdu words.

In [7]:
file = codecs.open("Data/Common_Words.txt", "r",encoding="utf-8")
dontRemove = []
for line in file:
    dontRemove.append(line.strip())
file.close()

Sorted_Words.txt: A list 14 lac Urdu words sorted (max to min) on their frequency of occurrence. 

In [8]:
file = codecs.open("Data/Sorted_Words.txt", "r",encoding="utf-8")
sortedMaxToMinWords = []
for line in file:
    sortedMaxToMinWords.append(line.strip())
file.close()

##  Make predictions

Let's simplify a sentence to see how it works.

In [9]:
complex_sentence = ['میرے بلانے پر وہ ٹھٹھک کر مڑے']
simple_sentence = simplify(complex_sentence, synModel, tagger, lm, sortedMaxToMinWords, dontRemove)
print("Complex Sentence: " + str(complex_sentence[0]))
print("Simple Sentence: " + str(simple_sentence[0]))

Complex Sentence: میرے بلانے پر وہ ٹھٹھک کر مڑے
Simple Sentence: میرے بلانے پر وہ چونک کر مڑے


## Evaluate the model

Simplify all sentences in our test set. Complex.txt contains 500 complex sentences taken from newspapers, magazines, books and literary journals.

In [10]:
file = codecs.open("Data/Complex.txt",'r',encoding='utf-8')
content = file.read()
content.replace(u'\ufeff','')
content = content.split('--')
i = 0
for c in content:
    content[i] = c.strip()
    i += 1
content = content[0:len(content)-1]
file.close()

In [11]:
final = simplify(content, synModel, tagger, lm, sortedMaxToMinWords, dontRemove)

Save the system output in a file.

In [12]:
file = codecs.open("System_Output.txt", 'w', encoding="utf-8")
for f in final:
    file.write(f.strip()+"\n"+"\n")
file.close()

Load the system output.

In [13]:
file = codecs.open("System_Output.txt",'r',encoding='utf-8')
candidates = file.read()
candidates.replace(u'\ufeff','')
candidates = candidates.split('\n\n')
i = 0
for c in candidates:
    candidates[i] = c.strip()
    i += 1
candidates = candidates[0:len(candidates)-1]
file.close()

Load the reference simplifications. The simplifications were created by an expert linguist.

In [14]:
file = codecs.open("Data/Simplified.txt",'r',encoding='utf-8')
references = file.read()
references.replace(u'\ufeff','')
references = references.split('--')
references = references[0:len(references)-1]
ind = 0
for ref in references:
    references[ind] = ref.strip().split('\r\n')
    ind += 1
file.close()

Calculate BLEU score.

In [15]:
references_words = copy.deepcopy(references)

i = 0
for ref in references_words:
    ind = 0
    for r in ref:
        references_words[i][ind] = r.split(' ')
        ind += 1
    i += 1

average = 0
index = 0
cc = SmoothingFunction()
for candidate in candidates:
    BLEU = sentence_bleu(references_words[index], candidate.split(' '), weights=(1, 0, 0, 0), smoothing_function=cc.method1)
    average += BLEU
    index += 1

den = len(candidates)
print("Average BLEU Score: " + "{:.2f}".format((average/den)*100))

Average BLEU Score: 80.15


Calculate SARI score. Implementation taken form https://github.com/cocoxu/simplification/blob/master/SARI.py

In [16]:
def ReadInFile(filename):
    with open(filename) as f:
        lines = f.readlines()
        lines = [x.strip() for x in lines]
    return lines


def SARIngram(sgrams, cgrams, rgramslist, numref):
    rgramsall = [rgram for rgrams in rgramslist for rgram in rgrams]
    rgramcounter = Counter(rgramsall)

    sgramcounter = Counter(sgrams)
    sgramcounter_rep = Counter()
    for sgram, scount in sgramcounter.items():
        sgramcounter_rep[sgram] = scount * numref

    cgramcounter = Counter(cgrams)
    cgramcounter_rep = Counter()
    for cgram, ccount in cgramcounter.items():
        cgramcounter_rep[cgram] = ccount * numref

    # KEEP
    keepgramcounter_rep = sgramcounter_rep & cgramcounter_rep
    keepgramcountergood_rep = keepgramcounter_rep & rgramcounter
    keepgramcounterall_rep = sgramcounter_rep & rgramcounter

    keeptmpscore1 = 0
    keeptmpscore2 = 0
    for keepgram in keepgramcountergood_rep:
        keeptmpscore1 += keepgramcountergood_rep[keepgram] / keepgramcounter_rep[keepgram]
        keeptmpscore2 += keepgramcountergood_rep[keepgram] / keepgramcounterall_rep[keepgram]
        # print "KEEP", keepgram, keepscore, cgramcounter[keepgram], sgramcounter[keepgram], rgramcounter[keepgram]
    keepscore_precision = 0
    if len(keepgramcounter_rep) > 0:
        keepscore_precision = keeptmpscore1 / len(keepgramcounter_rep)
    keepscore_recall = 0
    if len(keepgramcounterall_rep) > 0:
        keepscore_recall = keeptmpscore2 / len(keepgramcounterall_rep)
    keepscore = 0
    if keepscore_precision > 0 or keepscore_recall > 0:
        keepscore = 2 * keepscore_precision * keepscore_recall / (keepscore_precision + keepscore_recall)

    # DELETION
    delgramcounter_rep = sgramcounter_rep - cgramcounter_rep
    delgramcountergood_rep = delgramcounter_rep - rgramcounter
    delgramcounterall_rep = sgramcounter_rep - rgramcounter
    deltmpscore1 = 0
    deltmpscore2 = 0
    for delgram in delgramcountergood_rep:
        deltmpscore1 += delgramcountergood_rep[delgram] / delgramcounter_rep[delgram]
        deltmpscore2 += delgramcountergood_rep[delgram] / delgramcounterall_rep[delgram]
    delscore_precision = 0
    if len(delgramcounter_rep) > 0:
        delscore_precision = deltmpscore1 / len(delgramcounter_rep)
    delscore_recall = 0
    if len(delgramcounterall_rep) > 0:
        delscore_recall = deltmpscore1 / len(delgramcounterall_rep)
    delscore = 0
    if delscore_precision > 0 or delscore_recall > 0:
        delscore = 2 * delscore_precision * delscore_recall / (delscore_precision + delscore_recall)

    # ADDITION
    addgramcounter = set(cgramcounter) - set(sgramcounter)
    addgramcountergood = set(addgramcounter) & set(rgramcounter)
    addgramcounterall = set(rgramcounter) - set(sgramcounter)

    addtmpscore = 0
    for addgram in addgramcountergood:
        addtmpscore += 1

    addscore_precision = 0
    addscore_recall = 0
    if len(addgramcounter) > 0:
        addscore_precision = addtmpscore / len(addgramcounter)
    if len(addgramcounterall) > 0:
        addscore_recall = addtmpscore / len(addgramcounterall)
    addscore = 0
    if addscore_precision > 0 or addscore_recall > 0:
        addscore = 2 * addscore_precision * addscore_recall / (addscore_precision + addscore_recall)

    return (keepscore, delscore_precision, addscore)


def SARIsent(ssent, csent, rsents):
    numref = len(rsents)

    s1grams = ssent.lower().split(" ")
    c1grams = csent.lower().split(" ")
    s2grams = []
    c2grams = []
    s3grams = []
    c3grams = []
    s4grams = []
    c4grams = []

    r1gramslist = []
    r2gramslist = []
    r3gramslist = []
    r4gramslist = []
    for rsent in rsents:
        r1grams = rsent.lower().split(" ")
        r2grams = []
        r3grams = []
        r4grams = []
        r1gramslist.append(r1grams)
        for i in range(0, len(r1grams) - 1):
            if i < len(r1grams) - 1:
                r2gram = r1grams[i] + " " + r1grams[i + 1]
                r2grams.append(r2gram)
            if i < len(r1grams) - 2:
                r3gram = r1grams[i] + " " + r1grams[i + 1] + " " + r1grams[i + 2]
                r3grams.append(r3gram)
            if i < len(r1grams) - 3:
                r4gram = r1grams[i] + " " + r1grams[i + 1] + " " + r1grams[i + 2] + " " + r1grams[i + 3]
                r4grams.append(r4gram)
        r2gramslist.append(r2grams)
        r3gramslist.append(r3grams)
        r4gramslist.append(r4grams)

    for i in range(0, len(s1grams) - 1):
        if i < len(s1grams) - 1:
            s2gram = s1grams[i] + " " + s1grams[i + 1]
            s2grams.append(s2gram)
        if i < len(s1grams) - 2:
            s3gram = s1grams[i] + " " + s1grams[i + 1] + " " + s1grams[i + 2]
            s3grams.append(s3gram)
        if i < len(s1grams) - 3:
            s4gram = s1grams[i] + " " + s1grams[i + 1] + " " + s1grams[i + 2] + " " + s1grams[i + 3]
            s4grams.append(s4gram)

    for i in range(0, len(c1grams) - 1):
        if i < len(c1grams) - 1:
            c2gram = c1grams[i] + " " + c1grams[i + 1]
            c2grams.append(c2gram)
        if i < len(c1grams) - 2:
            c3gram = c1grams[i] + " " + c1grams[i + 1] + " " + c1grams[i + 2]
            c3grams.append(c3gram)
        if i < len(c1grams) - 3:
            c4gram = c1grams[i] + " " + c1grams[i + 1] + " " + c1grams[i + 2] + " " + c1grams[i + 3]
            c4grams.append(c4gram)

    (keep1score, del1score, add1score) = SARIngram(s1grams, c1grams, r1gramslist, numref)
    (keep2score, del2score, add2score) = SARIngram(s2grams, c2grams, r2gramslist, numref)
    (keep3score, del3score, add3score) = SARIngram(s3grams, c3grams, r3gramslist, numref)
    (keep4score, del4score, add4score) = SARIngram(s4grams, c4grams, r4gramslist, numref)
    avgkeepscore = sum([keep1score, keep2score, keep3score, keep4score]) / 4
    avgdelscore = sum([del1score, del2score, del3score, del4score]) / 4
    avgaddscore = sum([add1score, add2score, add3score, add4score]) / 4
    finalscore = (avgkeepscore + avgdelscore + avgaddscore) / 3

    return finalscore

In [17]:
average = 0
i = 0
SARI = 0

for s in content:
    SARI = SARIsent(s, candidates[i], references[i])
    average += SARI
    i += 1

print("Average SARI Score: " + "{:.2f}".format((average/len(content))*100))

Average SARI Score: 42.02
